In [1]:
import pandas as pd
import xlrd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
#return outliers > mean+3sigma or < mean-3sigma
def outlier_3sigma(pd_column):
    mean_vals = pd_column.mean()
    std_vals = pd_column.std()
    min_value = mean_vals-3*std_vals
    max_value = mean_vals+3*std_vals
    outliers = np.nonzero((pd_column.values>max_value)|(pd_column.values<min_value))
    return outliers

In [3]:
def run_columns(pd_column,idx2):
    for col_header in pd_column.columns:
        idx1 = outlier_3sigma(pd_column[col_header])
        idx2 = np.union1d(idx1,idx2)
        idx2 = np.unique(idx2)
    return idx2

## DEFINING VARIABLES 

**dfNormal_without_outlier** --- Variable without outlier with full charge of refrigerant -- y =0

**df_without_outlier** --- Variable without outlier with 40% refrigerant leaking --- y = 1

Joining dataset "x"

In [4]:
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Refrigerant leak/rl40.xls"
sheet = "Complete Data Set"
leaking_40 = pd.read_excel(path_to_xls,sheet)
idx2 = np.array([])
idx2 = run_columns(leaking_40,idx2)
df_without_outlier= leaking_40.drop(idx2)
path_to_xls = "F:/Talles/Doutorado/database/1043-RP/FDD Data/Benchmark Tests/normal r.xls"
sheet = "Complete Data Set"
Normal_R = pd.read_excel(path_to_xls,sheet)
idx2 = np.array([])
idx2 = run_columns(Normal_R,idx2)
dfNormal_without_outlier = Normal_R.drop(idx2)
# reseting index
df_without_outlier = df_without_outlier.reset_index(drop=True)
dfNormal_without_outlier = dfNormal_without_outlier.reset_index(drop=True)

In [5]:
# adding column with constant value
# 0 for no leak, 1 for leaking
dfNormal_without_outlier['y'] = pd.Series([0 for x in range(len(dfNormal_without_outlier.index))])
df_without_outlier['y'] = pd.Series([1 for x in range(len(df_without_outlier.index))])
# concatenating dataframes
data = dfNormal_without_outlier.append(df_without_outlier, ignore_index=True, sort=False)

In [6]:
data = np.asarray(data)
x,y = (data[:,:-1],data[:,-1])
print(x.shape,y.shape)

(9318, 66) (9318,)


In [7]:
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6243, 66) (3075, 66) (6243,) (3075,)


In [8]:
# standardscaler x data
sc_x = StandardScaler()
X_trainScaled = sc_x.fit_transform(X_train)
X_testScaled = sc_x.fit_transform(X_test)

In [9]:
clf = MLPClassifier(hidden_layer_sizes=(256,128,64,32),activation="relu",random_state=1).fit(X_trainScaled, y_train)
y_pred=clf.predict(X_testScaled)
print(clf.score(X_testScaled, y_test))

0.999349593495935
